## 참고 문서
- https://wikidocs.net/31766

### 전처리 (NLTK)

In [8]:
from pprint import pprint
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords

stopwords_set = set(stopwords.words('english'))

In [12]:
raw = "A barber is a person. a barber is good person. a barber is huge person. he Knew A Secret! The Secret He Kept is huge secret. Huge secret. His barber kept his word. a barber kept his word. His barber kept his secret. But keeping and keeping such a huge secret to himself was driving the barber crazy. the barber went up a huge mountain."
pprint(raw)

sentences = sent_tokenize(raw)
pprint(sentences)

('A barber is a person. a barber is good person. a barber is huge person. he '
 'Knew A Secret! The Secret He Kept is huge secret. Huge secret. His barber '
 'kept his word. a barber kept his word. His barber kept his secret. But '
 'keeping and keeping such a huge secret to himself was driving the barber '
 'crazy. the barber went up a huge mountain.')
['A barber is a person.',
 'a barber is good person.',
 'a barber is huge person.',
 'he Knew A Secret!',
 'The Secret He Kept is huge secret.',
 'Huge secret.',
 'His barber kept his word.',
 'a barber kept his word.',
 'His barber kept his secret.',
 'But keeping and keeping such a huge secret to himself was driving the barber '
 'crazy.',
 'the barber went up a huge mountain.']


In [20]:
docs = []

for sentence in sentences:
    tokens = word_tokenize(sentence)

    doc = []
    
    for token in tokens:
        word = token.lower()
        
        if word not in stopwords_set and len(word) > 2:
            doc.append(word)
            
    docs.append(doc)

In [21]:
pprint(docs)

[['barber', 'person'],
 ['barber', 'good', 'person'],
 ['barber', 'huge', 'person'],
 ['knew', 'secret'],
 ['secret', 'kept', 'huge', 'secret'],
 ['huge', 'secret'],
 ['barber', 'kept', 'word'],
 ['barber', 'kept', 'word'],
 ['barber', 'kept', 'secret'],
 ['keeping', 'keeping', 'huge', 'secret', 'driving', 'barber', 'crazy'],
 ['barber', 'went', 'huge', 'mountain']]


### 케라스를 이용한 정수 인코딩

In [30]:
from tensorflow.keras.preprocessing.text import Tokenizer

vocap_size = 20 # 상위 N 개 단어만 이용
keras_tokenizer = Tokenizer(num_words = vocap_size + 1)
keras_tokenizer.fit_on_texts(docs) # 빈도 기준으로 단어 집합을 생성

In [31]:
pprint(keras_tokenizer.word_index)

{'barber': 1,
 'crazy': 11,
 'driving': 10,
 'good': 8,
 'huge': 3,
 'keeping': 7,
 'kept': 4,
 'knew': 9,
 'mountain': 13,
 'person': 5,
 'secret': 2,
 'went': 12,
 'word': 6}


In [32]:
pprint(keras_tokenizer.word_counts)

OrderedDict([('barber', 8),
             ('person', 3),
             ('good', 1),
             ('huge', 5),
             ('knew', 1),
             ('secret', 6),
             ('kept', 4),
             ('word', 2),
             ('keeping', 2),
             ('driving', 1),
             ('crazy', 1),
             ('went', 1),
             ('mountain', 1)])


In [33]:
pprint(keras_tokenizer.texts_to_sequences(docs))

[[1, 5],
 [1, 8, 5],
 [1, 3, 5],
 [9, 2],
 [2, 4, 3, 2],
 [3, 2],
 [1, 4, 6],
 [1, 4, 6],
 [1, 4, 2],
 [7, 7, 3, 2, 10, 1, 11],
 [1, 12, 3, 13]]


### 원-핫 인코딩

In [46]:
from tensorflow.keras.utils import to_categorical

word_indice = keras_tokenizer.word_index
word_sequences = keras_tokenizer.texts_to_sequences(docs)

pprint(word_sequences)

[[1, 5],
 [1, 8, 5],
 [1, 3, 5],
 [9, 2],
 [2, 4, 3, 2],
 [3, 2],
 [1, 4, 6],
 [1, 4, 6],
 [1, 4, 2],
 [7, 7, 3, 2, 10, 1, 11],
 [1, 12, 3, 13]]


In [49]:
to_categorical(word_sequences[1])

array([[0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0.]], dtype=float32)

### 원-핫 인코딩의 한계

- 단어의 개수가 늘어날수록 벡터를 위한 공간 (메모리) 가 많이 필요 = Sparse
- 또한 단어간 유사도를 표현하지 못함.

이러한 단점을 해결하기 위해 단어의 잠재 의미를 반영하여, 다차원 공간에 벡터화 하는 기법으로 크게 두 가지가 존재

1. 카운트 기반의 벡터화 방법인 LSA, HAL
2. 예측 기반으로 벡터화 하는 NNLM, RNNLM, Word2Vec, FastText 등